# Open-Source Software PiTCT for Supervisory Control Design

## Part 2: Supervisory Control Synthesis

In this notebook, we demonstrate the functionality of PiTCT for supervisory control synthesis.  
We will follow three main steps:
1.  **Modeling**: Define the plant with controllable ('c') and uncontrollable ('u') events.
2.  **Specification**: Define what we want the system to do.
3.  **Synthesis**: Automatically generate a supervisor (if one exists) that enforces the specification.

## Printer Example (modified)

We create an automaton named `PRINTER` that models the high-level behavior of a simple printer. The system has 4 states:

- **State 0 (IDLE)**: The printer is ready and waiting.
- **State 1 (WORKING)**: The printer is currently printing.
- **State 2 (BROKEN)**: The printer has malfunctioned and needs repair.
- **State 3 (SOLD)**: The printer is sold.

State transitions are triggered by event occurrences:

- **start (controllable)**: Start a printing job, transitioning from IDLE to WORKING.
- **auto_finish (uncontrollable)**: Complete a printing job successfully, transitioning from WORKING to IDLE.
- **manual_stop (controllable)**: Stop a printing job manually, transitioning from WORKING to IDLE.
- **breakdown (uncontrollable)**: Break down (e.g. paper jam), transitioning from WORKING to BROKEN.
- **fix (controllable)**: Fix the breakdown problem, transitioning from BROKEN to IDLE.
- **sell (controllable)**: Sell the broken printer, transitioning from BROKEN to SOLD.

**Initial State**: States 0 (IDLE)

**Marker States**: States 0 (IDLE), 1 (WORKING), and 3 (SOLD) are marked as "acceptable" states.  

### 0. Prepare PiTCT (installation and initialization as in part 1)

In [1]:
%pip install pitct

In [2]:
import pitct
pitct.init("printer_sct", overwrite=True)

### 1. Plant Modeling with Control Attributes

We define the `PRINTER_SELL` automaton.

- **Controllable ('c')**: Events the supervisor can disable (e.g., `start`, `manual_stop`, `fix`, `sell`).
- **Uncontrollable ('u')**: Events that happen spontaneously and cannot be prevented (e.g., `breakdown`, `auto_finish`).

In PiTCT visualization, controllable transitions are **Red**, and uncontrollable ones are **Green** if you use `color=True` option.

In [3]:
# number of states
# states are sequentially labeled 0,1,...,statenum-1
# initial state is labeled 0
statenum = 4 

# set of transitions
# each triple is (exit state, event label, entering state, event type)
# each event type is either 'c' (controllable) or 'u' (uncontrollable)
trans=[(0, "start", 1, "c"),
       (1, "auto_finish", 0, "u"),  # Uncontrollable!
       (1, "manual_stop", 0, "c"),
       (1, "breakdown", 2, "u"),    # Uncontrollable!
       (2, "fix", 0, "c"),
       (2, "sell", 3, "c")]

# set of marker states
marker = [0, 1, 3]

# create automaton PRINTER_SELL
pitct.create('PRINTER_SELL', statenum, trans, marker)
pitct.display_automaton('PRINTER_SELL', color=True)

### 2. Specifications and Controllability

We define two different types of specifications and check if they are "Controllable" wrt. the plant (i.e., enforcible without disabling uncontrollable events).

#### State-based Specification S1: Using Subautomaton Function

We remove State 3 (Sell) and the `manual_stop` transition.

In [4]:
# Create S1 by removing:
# 1. State [3] -> Prevents reaching 'SOLD'
# 2. Transition [(1, "manual_stop", 0)] -> Prevents 'manual_stop' action
pitct.subautomaton("S1", "PRINTER_SELL", [3], [(1, "manual_stop", 0)])

# plot S1.DES with color coding
pitct.display_automaton("S1",color=True)

/lib/python3.13/site-packages/pitct/des_check.py:65: UserWarning: Too many number of state. It is recommend to set 3
  warnings.warn(f"Too many number of state. It is recommend to set {max_state}")


In [5]:
# Check Controllability
s1_controllable = pitct.is_controllable('PRINTER_SELL', 'S1')
print(f"Is S1 controllable? -> {s1_controllable}")

Is S1 controllable? -> True


#### Trajectory-based Specification S2: Using Create Function

For this type, we define a **new automaton from scratch**.

Unlike S1, **S2 is NOT a subautomaton** of the plant. It defines a logic where only one breakdown is allowed:
- **State 0**: No breakdown has occurred yet.
- **State 1**: One breakdown has occurred.
- **Note**: There is NO `breakdown` transition defined from State 1. This implies that a second breakdown is forbidden.

In [6]:
# number of states
# states are sequentially labeled 0,1,...,statenum
# initial state is labeled 0
statenum = 2

# set of transitions
# each triple is (exit state, event label, entering state)
# each event is either 'c' (controllable) or 'u' (uncontrollable)
trans=[(0, "start", 0, "c"),
       (0, "auto_finish", 0, "u"),
       (0, "manual_stop", 0, "c"),
       (0, "breakdown", 1, "u"),
       (1, "start", 1, "c"),
       (1, "auto_finish", 1, "u"),
       (1, "manual_stop", 1, "c"),
       (1, "fix", 1, "c"),
       (1, "sell", 1, "c")]

# set of marker states
marker = [0,1]

# create automaton S2
pitct.create('S2', statenum, trans, marker)

# plot S2.DES with color coding
pitct.display_automaton('S2',color=True)

In [7]:
# Check Controllability
is_s2_ctrl = pitct.is_controllable("PRINTER_SELL", "S2")
print(f"Is S2 controllable? -> {is_s2_ctrl}")

Is S2 controllable? -> False


**Interpretation of Uncontrollability:**

The following function identifies those states where the **Plant** can execute an uncontrollable event (like `breakdown`), but the **Specification (S2)** does not allow it. Since the supervisor cannot disable `breakdown` ('u'), the specification S2 is deemed uncontrollable.

In [8]:
print("Uncontrollable states")
pitct.uncontrollable_states("PRINTER_SELL", "S2")

Uncontrollable states


[1]

### 3. Supervisor Synthesis

When a specification is not controllable (like S2), we cannot enforce it in full by a supervisor.
So we use the **`supcon`** function to compute the **Supremal Controllable Sublanguage** of the specification language (confined by the plant language).

#### Synthesis for S1 (Controllable Case)
Since S1 was already verified to be controllable, the synthesized supervisor `C1` effectively retains the same behavior.  
`C1` is isomorphic to the trimmed version of `S1`.

In [9]:
# compute optimal supervisor
pitct.supcon("C1", "PRINTER_SELL", "S1")

# Verify that C1 is isomorphic to S1 (after trimming S1)
pitct.trim("S1_trim", "S1")
pitct.isomorph("C1", "S1_trim")

True

In [10]:
pitct.display_automaton("C1", color=True)

#### Synthesis for S2 (Uncontrollable Case)

For S2, which is not controllable, `supcon` computes the **optimal supervisor** `C2`.  
This supervisor disables the `start` event at the state following a repair (State 3) to prevent a second breakdown.

In [11]:
# Compute Supervisor C2
pitct.supcon("C2", "PRINTER_SELL", "S2")
pitct.display_automaton("C2", color=True)

### Inspecting Control Actions

We check which controllable events that the supervisor disables at which states.



In [12]:
# compute supervisor’s control actions
pitct.conact("PRINTER_SELL", "C2")

'3: start'

**Result:** controllable event **`start`** is disabled at **State 3**. This prevents the printer from restarting a job that could lead to a second breakdown.